<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Transports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
global department_sequence_counters
department_sequence_counters = {}

def Transport_create(list_of_shipments_df):
    """
    Creates a Transport object from a DataFrame of shipments.
    The transport will have a unique ID: {Department}A-XXXX (sequential per department).

    Args:
        list_of_shipments_df (pd.DataFrame): A DataFrame containing shipment details.

    Returns:
        Transport: A newly created Transport object.

    Raises:
        ValueError: If the input DataFrame is empty or if sequential IDs for a department are exhausted.
    """
    if list_of_shipments_df.empty:
        raise ValueError("Input DataFrame is empty. Cannot create a Transport object.")

    # Get department from the first shipment
    department = list_of_shipments_df.iloc[0]['Department']

    # Get a list of Shipment_IDs for the Transport object
    Shipment_IDs = list_of_shipments_df['Shipment_ID'].tolist()

    # Calculate Pickup_date (earliest date) and Delivery_date (latest date)
    # Ensure dates are in datetime format for proper min/max calculation
    earliest_pickup_date = pd.to_datetime(list_of_shipments_df['Pickup_date']).min().date()
    latest_delivery_date = pd.to_datetime(list_of_shipments_df['Delivery_date']).max().date()

    # Calculate combined Weight, Volume, and Ldm
    total_weight = list_of_shipments_df['Weight'].sum()
    total_volume = list_of_shipments_df['Volume'].sum()
    total_ldm = list_of_shipments_df['Ldm'].sum()

    #create a beginning value, same for all transports for the following.
    tp_status = "Planning"
    tp_vehicle = ""
    tp_haulier = ""
    tp_driver = ""
    # Ensure the department has an entry in the sequence counter
    if department not in department_sequence_counters:
        department_sequence_counters[department] = 0 # Start from 0, so first increment makes it 1

    # Generate a unique sequential Transport ID
    Transport_ID = None
    max_sequence_value = 9999

    # Iterate to find the next available sequential ID for the department
    while True:
        department_sequence_counters[department] += 1
        sequence_num = department_sequence_counters[department]

        # Check if the sequence number has exceeded the allowed range (0001-9999)
        if sequence_num > max_sequence_value:
            raise ValueError(f"Exceeded maximum sequential IDs ({max_sequence_value}) for department '{department}'.")

        formatted_sequence = f"{sequence_num:04d}"
        proposed_id = f"{department}A-{formatted_sequence}"

        # Check if this proposed ID is already in use in the Transport class's registry
        if Transport.get_by_id(proposed_id) is None:
            Transport_ID = proposed_id
            break

    # Create the Transport object using the generated unique ID and calculated dates
    new_transport = Transport(Transport_ID, department, Shipment_IDs, earliest_pickup_date, latest_delivery_date, total_weight, total_volume, total_ldm, tp_status, tp_vehicle, tp_haulier, tp_driver)


    # Update the Transport_ID for each shipment object
    for Shipment_ID in Shipment_IDs:
        shipment_obj = Shipment.get_by_id(Shipment_ID)
        if shipment_obj:
            shipment_obj.Transport = new_transport.Transport_ID

    print(f"Department set to {department} and Created Transport object with ID: {new_transport.Transport_ID}")
    return new_transport

In [ ]:
def Transport_add(transport_id, list_of_shipments_df):
    """
    Adds shipments from a DataFrame to an existing Transport object.
    Only shipments with 'Transport' set to None are considered.
    Recalculates Weight, Volume, Ldm, Pickup_date (earliest), and Delivery_date (latest)
    for the Transport object.

    Args:
        transport_id (str): The ID of the Transport object to add shipments to.
        list_of_shipments_df (pd.DataFrame): DataFrame containing shipment details.

    Returns:
        Transport: The updated Transport object, or None if not found.
    """
    transport_obj = Transport.get_by_id(transport_id)
    if not transport_obj:
        print(f"Error: Transport with ID '{transport_id}' not found.")
        return None

    # Filter shipments where 'Transport' is None (or equivalent string 'None')
    # Convert all 'Transport' column values to string for consistent comparison
    filtered_shipments_df = list_of_shipments_df[
        list_of_shipments_df['Transport'].astype(str) == 'None'
    ].copy() # Use .copy() to avoid SettingWithCopyWarning

    if filtered_shipments_df.empty:
        print(f"No new shipments with Transport=None to add to {transport_id}.")
        return transport_obj

    # Initialize accumulation variables with current transport object's values
    current_weight = transport_obj.Weight
    current_volume = transport_obj.Volume
    current_ldm = transport_obj.Ldm


    added_shipment_ids = []

    for index, row in filtered_shipments_df.iterrows():
        shipment_id = row['Shipment_ID']
        shipment_obj = Shipment.get_by_id(shipment_id)

        # Ensure shipment_obj exists and is not already assigned to this transport
        # Also check if it's not already in the transport_obj.Shipments list
        if shipment_obj and shipment_obj.Transport != transport_id and shipment_id not in transport_obj.Shipments:
            # Add shipment ID to Transport object's list
            transport_obj.Shipments.append(shipment_id)

            # Update individual Shipment object's Transport attribute
            shipment_obj.Transport = transport_id

            # Accumulate totals
            current_weight += row['Weight']
            current_volume += row['Volume']
            current_ldm += row['Ldm']

            # Dates are no longer updated as per user request.
            # shipment_pickup_date = pd.to_datetime(row['Pickup_date']).date()
            # shipment_delivery_date = pd.to_datetime(row['Delivery_date']).date()

            # if shipment_pickup_date < current_pickup_date:
            #     current_pickup_date = shipment_pickup_date
            # if shipment_delivery_date > current_delivery_date:
            #     current_delivery_date = shipment_delivery_date

            added_shipment_ids.append(shipment_id)

    # Update the Transport object's attributes
    transport_obj.Weight = current_weight
    transport_obj.Volume = current_volume
    transport_obj.Ldm = current_ldm


    if added_shipment_ids:
        print(f"Successfully added shipments {added_shipment_ids} to Transport '{transport_obj.Transport_ID}'.")
        print(f"Updated Transport details: Weight={transport_obj.Weight}, Volume={transport_obj.Volume}, Ldm={transport_obj.Ldm},")
        print(f"  Pickup_date={transport_obj.Pickup_date}, Delivery_date={transport_obj.Delivery_date}")
    else:
        print(f"No new shipments were added to Transport '{transport_obj.Transport_ID}'.")

    return transport_obj

In [ ]:
def Transport_remove(transport_id, shipment_ids):
    """
    Removes one or more shipments from a transport and updates the transport's metrics.

    Args:
        transport_id (str): The ID of the transport from which to remove the shipment(s).
        shipment_ids (list): A list of Shipment IDs to remove.

    Returns:
        Transport: The updated Transport object, or None if the transport was not found.
    """
    transport = Transport.get_by_id(transport_id)
    if not transport:
        print(f"Error: Transport with ID '{transport_id}' not found.")
        return None

    removed_count = 0
    for shipment_id in shipment_ids:
        if shipment_id not in transport.Shipments:
            print(f"Shipment '{shipment_id}' is not part of Transport '{transport_id}'. Skipping.")
            continue

        # 4a. Remove the shipment_id from the Transport object's Shipments list.
        transport.Shipments.remove(shipment_id)

        # 4b. Retrieve the corresponding Shipment object
        shipment_to_remove = Shipment.get_by_id(shipment_id)
        if not shipment_to_remove:
            print(f"Warning: Shipment object '{shipment_id}' not found in registry, but proceeding with transport update.")
        else:
            # 4c. Set its Transport attribute to None
            shipment_to_remove.Transport = None

            # 4d. Subtract the Weight, Volume, and Ldm of the removed Shipment object
            transport.Weight -= shipment_to_remove.Weight
            transport.Volume -= shipment_to_remove.Volume
            transport.Ldm -= shipment_to_remove.Ldm
        removed_count += 1

    if removed_count > 0:
        # 4e. Print a confirmation message
        print(f"Successfully removed {removed_count} shipment(s) from Transport '{transport_id}'.")
        print(f"Updated Transport '{transport_id}' details: Weight={transport.Weight}, Volume={transport.Volume}, Ldm={transport.Ldm}")
    else:
        print(f"No shipments from the provided list were removed from Transport '{transport_id}'.")

    # 5. Return the updated Transport object.
    return transport

print("Transport_remove function defined and updated to handle multiple shipments.")